In [27]:
import pandas as pd
import itertools
import pulp


In [28]:
import numpy as np

In [29]:
df = pd.read_csv('filtered3_df_full.csv')
pred = pd.read_csv('filtered3_prediction_cutoff_2.csv')
pred = pred[['PLAYER_NAME', 'PRED_SCORE']]
df.columns.values
df = df[['Season','Game_ID', 'PLAYER_NAME', 'POS', 'Team', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT',
       'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE', 'DD', 'TD',
       'total_fantasy_points', 'salary']]


In [4]:
# DO NOT RUN
def get_recent_salary(df):
    # sort by game date in descending order
    df_sorted = df.sort_values(by=['GAME_DATE'], ascending=False)
    
    # group by player name and get first row of each group to get player's most recent game 
    df_recent_salary = df_sorted.groupby('PLAYER_NAME').first()
    
    return df_recent_salary[['salary']].reset_index()

df_recent_salary = get_recent_salary(df)




In [45]:
# Example lineup for game date 2021-5-1
# RUN
df_ex = df[(df['GAME_DATE'] == '2021-05-01')]
merged_df = pd.merge(pred, df_ex[['PLAYER_NAME', 'salary', 'POS', 'total_fantasy_points']], on='PLAYER_NAME')
merged_df
merged_df.columns.values
merged_df = merged_df.drop_duplicates(subset=['PLAYER_NAME'], keep='first')
merged_df


,PLAYER_NAME,PRED_SCORE,salary,POS,total_fantasy_points
0,Alex Len,27.945639,3500.0,C,7.25
1,Bam Adebayo,31.262215,8800.0,C,40.00
2,Bismack Biyombo,17.388153,3100.0,C,30.75
3,Christian Wood,28.114052,8700.0,C,22.75
4,Clint Capela,33.878757,8600.0,C,30.25
...,...,...,...,...,...
78,Onyeka Okongwu,11.599199,3800.0,C,9.75
79,Precious Achiuwa,14.514090,3000.0,C,4.50
80,Saben Lee,11.509678,3000.0,PG,17.25
83,Saddiq Bey,22.949059,5200.0,SF,38.25


In [31]:
filtered_df = merged_df[merged_df['POS'] == 'PG']
min_value = filtered_df['salary'].min()
print(f"Minimum salary for PG: {min_value}")
least_salary_rows = filtered_df.nsmallest(5, 'salary')
print(least_salary_rows)
filtered_df = merged_df[merged_df['POS'] == 'SG']
min_value = filtered_df['salary'].min()
print(f"Minimum salary for SG: {min_value}")
least_salary_rows = filtered_df.nsmallest(5, 'salary')
print(least_salary_rows)
filtered_df = merged_df[merged_df['POS'] == 'SF']
min_value = filtered_df['salary'].min()
print(f"Minimum salary for SF: {min_value}")
least_salary_rows = filtered_df.nsmallest(5, 'salary')
print(least_salary_rows)
filtered_df = merged_df[merged_df['POS'] == 'PF']
min_value = filtered_df['salary'].min()
print(f"Minimum salary for PF: {min_value}")
least_salary_rows = filtered_df.nsmallest(5, 'salary')
print(least_salary_rows)
filtered_df = merged_df[merged_df['POS'] == 'C']
min_value = filtered_df['salary'].min()
print(f"Minimum salary for C: {min_value}")
least_salary_rows = filtered_df.nsmallest(5, 'salary')
print(least_salary_rows)


Minimum salary for PG: 3000.0
          PLAYER_NAME  PRED_SCORE  salary POS  total_fantasy_points
44   Ryan Arcidiacono   10.887293  3000.0  PG                  6.50
53         Tyus Jones   23.722527  3000.0  PG                 15.25
80          Saben Lee   11.509678  3000.0  PG                 17.25
16  Jordan McLaughlin   22.756399  3100.0  PG                  9.75
56      Austin Rivers   12.726202  3700.0  PG                 16.50
Minimum salary for SG: 3000.0
        PLAYER_NAME  PRED_SCORE  salary POS  total_fantasy_points
67     Gabe Vincent    8.748289  3000.0  SG                  5.75
86    Trent Forrest    8.375336  3000.0  SG                 15.00
66    Frank Jackson   17.354475  3900.0  SG                 29.75
76       Malik Monk   16.130264  4500.0  SG                 16.50
71  Josh Richardson   23.543966  4600.0  SG                 15.00
Minimum salary for SF: 4200.0
         PLAYER_NAME  PRED_SCORE  salary POS  total_fantasy_points
17    Justin Holiday   21.709976  4200.

In [32]:
# WITHOUT SALARY CONSTRAINT using pred score

def optimize_lineup(merged_df):
    # filter by positions
    PG = merged_df[merged_df['POS'] == 'PG']
    SG = merged_df[merged_df['POS'] == 'SG']
    SF = merged_df[merged_df['POS'] == 'SF']
    PF = merged_df[merged_df['POS'] == 'PF']
    C = merged_df[merged_df['POS'] == 'C']

    # define positions and # of players for each position
    positions = {'PG': 1, 'SG': 1, 'SF': 1, 'PF': 1, 'C': 1, 'G': 1, 'F': 1, 'Util': 1}

    # initialize the lineup dictionary
    lineup = {position: [] for position in positions.keys()}

    # loop through the sorted dataframes for each position
    for pos, df in {'PG': PG, 'SG': SG, 'SF': SF, 'PF': PF, 'C': C}.items():
        df = df.sort_values(by='PRED_SCORE', ascending=False)

        for idx, player in df.iterrows():
            if positions[pos] > 0 and sum(positions.values()) > 1:
                lineup[pos].append(player['PLAYER_NAME'])
                positions[pos] -= 1

    # fill the G, F, and Util spots
    all_positions_df = merged_df.sort_values(by='PRED_SCORE', ascending=False)
    for idx, player in all_positions_df.iterrows():
        pos = player['POS']

        if positions['G'] > 0 and pos in {'PG', 'SG'} and player['PLAYER_NAME'] not in lineup['PG'] + lineup['SG']:
            lineup['G'].append(player['PLAYER_NAME'])
            positions['G'] -= 1
        elif positions['F'] > 0 and pos in {'SF', 'PF'} and player['PLAYER_NAME'] not in lineup['SF'] + lineup['PF']:
            lineup['F'].append(player['PLAYER_NAME'])
            positions['F'] -= 1
        elif positions['Util'] > 0 and player['PLAYER_NAME'] not in (lineup['PG'] + lineup['SG'] + lineup['SF'] +
                                                                     lineup['PF'] + lineup['C'] + lineup['G'] +
                                                                     lineup['F']):
            lineup['Util'].append(player['PLAYER_NAME'])
            positions['Util'] -= 1

        if all(count == 0 for count in positions.values()):
            break

    total_score = sum(merged_df.loc[merged_df['PLAYER_NAME'].isin(lineup[pos]), 'PRED_SCORE'].sum() for pos in lineup.keys())
    total_salary = sum(merged_df.loc[merged_df['PLAYER_NAME'].isin(lineup[pos]), 'salary'].sum() for pos in lineup.keys())

    # print the lineup and the total score and total salary
    print('Lineup:')
    for position, players in lineup.items():
        print(f'{position}: {", ".join(players)}')
    print(f'Total score: {total_score}')
    print(f'Total salary: {total_salary}')

optimize_lineup(merged_df)


Lineup:
PG: Russell Westbrook
SG: Paul George
SF: Jimmy Butler
PF: Pascal Siakam
C: Rudy Gobert
G: T.J. McConnell
F: Kawhi Leonard
Util: Karl-Anthony Towns
Total score: 299.37929344177246
Total salary: 71900.0


In [33]:
# WITHOUT SALARY CONSTRAINT using total fantasy points

def optimize_lineup(merged_df):
    # filter by positions
    PG = merged_df[merged_df['POS'] == 'PG']
    SG = merged_df[merged_df['POS'] == 'SG']
    SF = merged_df[merged_df['POS'] == 'SF']
    PF = merged_df[merged_df['POS'] == 'PF']
    C = merged_df[merged_df['POS'] == 'C']

    # define positions and # of players for each position
    positions = {'PG': 1, 'SG': 1, 'SF': 1, 'PF': 1, 'C': 1, 'G': 1, 'F': 1, 'Util': 1}

    # initialize the lineup dictionary
    lineup = {position: [] for position in positions.keys()}

    # loop through the sorted dataframes for each position
    for pos, df in {'PG': PG, 'SG': SG, 'SF': SF, 'PF': PF, 'C': C}.items():
        df = df.sort_values(by='total_fantasy_points', ascending=False)

        for idx, player in df.iterrows():
            if positions[pos] > 0 and sum(positions.values()) > 1:
                lineup[pos].append(player['PLAYER_NAME'])
                positions[pos] -= 1

    # fill the G, F, and Util spots
    all_positions_df = merged_df.sort_values(by='total_fantasy_points', ascending=False)
    for idx, player in all_positions_df.iterrows():
        pos = player['POS']

        if positions['G'] > 0 and pos in {'PG', 'SG'} and player['PLAYER_NAME'] not in lineup['PG'] + lineup['SG']:
            lineup['G'].append(player['PLAYER_NAME'])
            positions['G'] -= 1
        elif positions['F'] > 0 and pos in {'SF', 'PF'} and player['PLAYER_NAME'] not in lineup['SF'] + lineup['PF']:
            lineup['F'].append(player['PLAYER_NAME'])
            positions['F'] -= 1
        elif positions['Util'] > 0 and player['PLAYER_NAME'] not in (lineup['PG'] + lineup['SG'] + lineup['SF'] +
                                                                     lineup['PF'] + lineup['C'] + lineup['G'] +
                                                                     lineup['F']):
            lineup['Util'].append(player['PLAYER_NAME'])
            positions['Util'] -= 1

        if all(count == 0 for count in positions.values()):
            break

    total_score = sum(merged_df.loc[merged_df['PLAYER_NAME'].isin(lineup[pos]), 'total_fantasy_points'].sum() for pos in lineup.keys())
    total_salary = sum(merged_df.loc[merged_df['PLAYER_NAME'].isin(lineup[pos]), 'salary'].sum() for pos in lineup.keys())

    # print the lineup and the total score and total salary
    print('Lineup:')
    for position, players in lineup.items():
        print(f'{position}: {", ".join(players)}')
    print(f'Total score: {total_score}')
    print(f'Total salary: {total_salary}')

optimize_lineup(merged_df)


Lineup:
PG: Russell Westbrook
SG: Caris LeVert
SF: Saddiq Bey
PF: Pascal Siakam
C: Karl-Anthony Towns
G: Cole Anthony
F: Kawhi Leonard
Util: Bam Adebayo
Total score: 289.0
Total salary: 67600.0


## Fixing merged_df[POS] column

In [46]:
merged_df.columns.values

array(['PLAYER_NAME', 'PRED_SCORE', 'salary', 'POS',
       'total_fantasy_points'], dtype=object)

In [47]:
merged_df['POS'].unique()

array(['C', 'PF', 'PG', 'SF', 'SG'], dtype=object)

In [48]:
pos_map = {'PF': 'PF, F, UTIL', 'C': 'C, UTIL', 'PG': 'PG, G, UTIL', 'SG': 'SG, G, UTIL', 'SF': 'SF, F, UTIL'}
merged_df['POS'] = merged_df['POS'].replace(pos_map)


In [49]:
import re

# Define a regular expression pattern to split the 'POS' column on commas
pattern = re.compile(r'\s*,\s*')

# Convert the 'POS' column to a tuple using the regular expression pattern to split the values
merged_df['POS'] = merged_df['POS'].apply(lambda x: tuple(pattern.split(x)))


In [50]:
merged_df

,PLAYER_NAME,PRED_SCORE,salary,POS,total_fantasy_points
0,Alex Len,27.945639,3500.0,"(C, UTIL)",7.25
1,Bam Adebayo,31.262215,8800.0,"(C, UTIL)",40.00
2,Bismack Biyombo,17.388153,3100.0,"(C, UTIL)",30.75
3,Christian Wood,28.114052,8700.0,"(C, UTIL)",22.75
4,Clint Capela,33.878757,8600.0,"(C, UTIL)",30.25
...,...,...,...,...,...
78,Onyeka Okongwu,11.599199,3800.0,"(C, UTIL)",9.75
79,Precious Achiuwa,14.514090,3000.0,"(C, UTIL)",4.50
80,Saben Lee,11.509678,3000.0,"(PG, G, UTIL)",17.25
83,Saddiq Bey,22.949059,5200.0,"(SF, F, UTIL)",38.25


In [39]:
merged_df.to_csv('merged_df_final.csv')

In [40]:
merged_df_final = pd.read_csv('merged_df_final.csv')

In [41]:
merged_df_final

,Unnamed: 0,PLAYER_NAME,PRED_SCORE,salary,POS,total_fantasy_points
0,0,Alex Len,27.945639,3500.0,"('C', 'UTIL')",7.25
1,1,Bam Adebayo,31.262215,8800.0,"('C', 'UTIL')",40.00
2,2,Bismack Biyombo,17.388153,3100.0,"('C', 'UTIL')",30.75
3,3,Christian Wood,28.114052,8700.0,"('C', 'UTIL')",22.75
4,4,Clint Capela,33.878757,8600.0,"('C', 'UTIL')",30.25
...,...,...,...,...,...,...
60,78,Onyeka Okongwu,11.599199,3800.0,"('C', 'UTIL')",9.75
61,79,Precious Achiuwa,14.514090,3000.0,"('C', 'UTIL')",4.50
62,80,Saben Lee,11.509678,3000.0,"('PG', 'G', 'UTIL')",17.25
63,83,Saddiq Bey,22.949059,5200.0,"('SF', 'F', 'UTIL')",38.25


# Memoization-based solution

In [42]:
def maximize_score(remaining_roles, current_cost, players, memo, lineup):
    if not remaining_roles:
        return 0, lineup

    if current_cost > 50000:
        return float('-inf'), lineup

    if not players:
        return float('-inf'), lineup

    state = (tuple(remaining_roles), current_cost)
    if state in memo:
        return memo[state]

    player = players[0]
    name, cost, predicted_score, valid_roles = player

    remaining_players = players[1:]

    # Calculate score by selecting the current player
    select_player_score, selected_lineup = float('-inf'), lineup
    if current_cost + cost <= 50000:
        for role in set(remaining_roles) & set(valid_roles):
            new_remaining_roles = list(remaining_roles)
            new_remaining_roles.remove(role)
            new_lineup = lineup.copy()
            new_lineup[role] = name
            score, lineup_result = maximize_score(new_remaining_roles, current_cost + cost, remaining_players, memo, new_lineup)
            if predicted_score + score > select_player_score:
                select_player_score = score + predicted_score
                selected_lineup = lineup_result

    # Calculate score by not selecting the current player
    skip_player_score, skip_lineup = maximize_score(remaining_roles, current_cost, remaining_players, memo, lineup)

    # Store the maximum score in the memoization cache
    if select_player_score > skip_player_score:
        memo[state] = (select_player_score, selected_lineup)
    else:
        memo[state] = (skip_player_score, skip_lineup)

    return memo[state]

In [43]:
# Load data from a CSV file
merged_df_2 = merged_df

# Set the 'PLAYER_NAME' column as the index
merged_df_2.set_index('PLAYER_NAME', inplace=True)

# Convert the dataframe to a list of tuples
players = merged_df_2.reset_index()[['PLAYER_NAME', 'salary', 'PRED_SCORE', 'POS']].apply(tuple, axis=1).tolist()

# Define the roles
roles = ["PG", "SG", "SF", "PF", "C", "G", "F", "UTIL"]

# Sort players by predicted score in descending order
sorted_players = sorted(players, key=lambda x: x[2], reverse=True)

# Initialize memoization cache
memo = {}

# Get the maximum predicted fantasy score
# Initialize an empty lineup
initial_lineup = {role: None for role in roles}

# Get the maximum predicted fantasy score and the lineup
max_score, optimal_lineup = maximize_score(roles, 0, sorted_players, memo, initial_lineup)
print("Maximum Predicted Fantasy Score:", max_score)
print("Optimal Lineup:", optimal_lineup)

# Calculate the total cost of the optimal lineup
total_cost = sum(merged_df_2.loc[optimal_lineup[role], 'salary'] for role in roles)
print("Total Cost of Optimal Lineup:", total_cost)


Maximum Predicted Fantasy Score: 244.1683464050293
Optimal Lineup: {'PG': 'T.J. McConnell', 'SG': 'Josh Richardson', 'SF': 'Justin Holiday', 'PF': 'JaMychal Green', 'C': 'Rudy Gobert', 'G': 'Paul George', 'F': 'Justise Winslow', 'UTIL': 'Russell Westbrook'}
Total Cost of Optimal Lineup: 50000.0


In [ ]:
df_ex = df[(df['GAME_DATE'] == '2021-05-01')]
merged_df = pd.merge(pred, df_ex[['PLAYER_NAME', 'salary', 'POS', 'total_fantasy_points']], on='PLAYER_NAME')
merged_df = merged_df.drop_duplicates(subset=['PLAYER_NAME'], keep='first')
merged_df['POS'].unique()
pos_map = {'PF': 'PF, F, UTIL', 'C': 'C, UTIL', 'PG': 'PG, G, UTIL', 'SG': 'SG, G, UTIL', 'SF': 'SF, F, UTIL'}
merged_df['POS'] = merged_df['POS'].replace(pos_map)
import re
# Define a regular expression pattern to split the 'POS' column on commas
pattern = re.compile(r'\s*,\s*')
# Convert the 'POS' column to a tuple using the regular expression pattern to split the values
merged_df['POS'] = merged_df['POS'].apply(lambda x: tuple(pattern.split(x)))
merged_df

In [51]:
# Load data from a CSV file
merged_df_2 = merged_df

# Set the 'PLAYER_NAME' column as the index
merged_df_2.set_index('PLAYER_NAME', inplace=True)

# Convert the dataframe to a list of tuples
players = merged_df_2.reset_index()[['PLAYER_NAME', 'salary', 'total_fantasy_points', 'POS']].apply(tuple, axis=1).tolist()

# Define the roles
roles = ["PG", "SG", "SF", "PF", "C", "G", "F", "UTIL"]

# Sort players by predicted score in descending order
sorted_players = sorted(players, key=lambda x: x[2], reverse=True)

# Initialize memoization cache
memo = {}

# Get the maximum predicted fantasy score
# Initialize an empty lineup
initial_lineup = {role: None for role in roles}

# Get the maximum predicted fantasy score and the lineup
max_score, optimal_lineup = maximize_score(roles, 0, sorted_players, memo, initial_lineup)
print("Maximum Predicted Fantasy Score:", max_score)
print("Optimal Lineup:", optimal_lineup)

# Calculate the total cost of the optimal lineup
total_cost = sum(merged_df_2.loc[optimal_lineup[role], 'salary'] for role in roles)
print("Total Cost of Optimal Lineup:", total_cost)


Maximum Predicted Fantasy Score: 277.5
Optimal Lineup: {'PG': 'Cole Anthony', 'SG': 'Frank Jackson', 'SF': 'Saddiq Bey', 'PF': 'Kenyon Martin Jr.', 'C': 'Karl-Anthony Towns', 'G': 'Ricky Rubio', 'F': 'Cody Martin', 'UTIL': 'Russell Westbrook'}
Total Cost of Optimal Lineup: 50000.0


# Greedy Algorithm

In [69]:
def greedy_maximize_score(remaining_roles, current_cost, players, lineup):
    total_predicted_score = 0

    for role in remaining_roles:
        best_player = None
        best_score = float('-inf')

        for i, player in enumerate(players):
            name, cost, predicted_score, valid_roles = player

            # Check if the current player can be selected for the current role
            if role in valid_roles and current_cost + cost <= 50000:
                if predicted_score > best_score:
                    best_player = i
                    best_score = predicted_score

        if best_player is not None:
            player = players.pop(best_player)
            name, cost, predicted_score, _ = player
            lineup[role] = name
            current_cost += cost
            total_predicted_score += predicted_score

    return total_predicted_score, lineup

# Load data from a CSV file
merged_df_2 = merged_df_final

# Set the 'PLAYER_NAME' column as the index
merged_df_2.set_index('PLAYER_NAME', inplace=True)

# Convert the dataframe to a list of tuples
players = merged_df_2.reset_index()[['PLAYER_NAME', 'salary', 'PRED_SCORE', 'POS']].apply(tuple, axis=1).tolist()

# Define the roles
roles = ["PG", "SG", "SF", "PF", "C", "G", "F", "UTIL"]

# Sort players by predicted score in descending order
sorted_players = sorted(players, key=lambda x: x[2], reverse=True)

# Initialize an empty lineup
initial_lineup = {role: None for role in roles}

# Get the maximum predicted fantasy score and the lineup
max_score, optimal_lineup = greedy_maximize_score(roles, 0, sorted_players, initial_lineup)
print("Maximum Predicted Fantasy Score:", max_score)
print("Optimal Lineup:", optimal_lineup)

# Calculate the total cost of the optimal lineup
total_cost = sum(merged_df_2.loc[optimal_lineup[role], 'salary'] for role in roles)
print("Total Cost of Optimal Lineup:", total_cost)


Maximum Predicted Fantasy Score: 92.02250719070435
Optimal Lineup: {'PG': 'Trae Young', 'SG': 'Max Strus', 'SF': 'Yuta Watanabe', 'PF': 'Kevin Durant', 'C': 'DeAndre Jordan', 'G': 'Landry Shamet', 'F': 'Aaron Gordon', 'UTIL': 'Christian Wood'}
Total Cost of Optimal Lineup: 49200.0
